<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week3/part1/3_02_Ejemplo_Memoria_Largo_Plazo_LangChain_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Memoria a Largo Plazo en Agentes (LangChain + LangGraph)

Colab-ready. Cubre: amnesia, corto vs largo, 4 pilares (Storage/Extraction/Indexing/Retrieval), técnicas de almacenamiento (relacional/vector/NoSQL/híbrido), implementación en LangChain, checkpointing en LangGraph, estrategia híbrida y mejores prácticas.


## 0) Setup


In [1]:

!pip -q install "langchain>=1,<2" langchain-core langchain-community
!pip -q install langchain-groq langsmith chromadb langgraph pydantic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [2]:

import os
from google.colab import userdata

if not os.getenv("GROQ_API_KEY"):
    try:
        os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
    except Exception:
        pass

print("GROQ_API_KEY:", "✅" if os.getenv("GROQ_API_KEY") else "⚠️")

GROQ_API_KEY: ✅


In [3]:

from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.2)
print("✅ LLM listo:", llm.model_name)


✅ LLM listo: llama-3.1-8b-instant


## 1) El problema de la amnesia (sin memoria persistente)


In [4]:

from langchain_core.messages import SystemMessage, HumanMessage

def one_shot(user_text: str):
    return llm.invoke([SystemMessage(content="Eres un asistente."), HumanMessage(content=user_text)]).content

print("Lunes:", one_shot("Me llamo Carlos y trabajo en marketing digital."))
print("Viernes:", one_shot("¿Qué cursos me recomiendas para mi profesión?"))


Lunes: Hola Carlos, un placer conocerte. ¿En qué puedo ayudarte hoy? ¿Tienes algún proyecto en mente que necesites asesorar o quieres discutir sobre tendencias en marketing digital?
Viernes: Para recomendarte cursos adecuados, necesito saber un poco más sobre tu profesión y objetivos. ¿Cuál es tu profesión actual o la que deseas desarrollar? ¿Cuáles son tus intereses y áreas de especialización?

Si me proporcionas esta información, puedo ofrecerte recomendaciones de cursos en línea o presenciales que se ajusten a tus necesidades y objetivos.

Algunas preguntas adicionales para ayudarme a entender mejor:

- ¿Cuál es tu nivel de experiencia en la profesión?
- ¿Qué habilidades o conocimientos deseas mejorar o desarrollar?
- ¿Cuál es tu presupuesto para cursos y capacitación?
- ¿Preferirías cursos en línea o presenciales?

Con esta información, puedo ofrecerte recomendaciones personalizadas para ayudarte a mejorar tus habilidades y alcanzar tus objetivos profesionales.


## 2) Corto vs Largo plazo

- **Corto**: historial de la sesión actual.
- **Largo**: hechos persistentes + recuerdos recuperables (días/semanas).


## 3) Los 4 pilares: Storage, Extraction, Indexing, Retrieval


## - Storage (hechos estructurados) con SQLite


In [5]:

import sqlite3, time, json

conn = sqlite3.connect("memoria_struct.db")
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS user_facts (
  user_id TEXT,
  key TEXT,
  value TEXT,
  ts INTEGER,
  PRIMARY KEY (user_id, key)
)
""")
conn.commit()

def upsert_fact(user_id: str, key: str, value: str):
    cur.execute(
        "INSERT OR REPLACE INTO user_facts(user_id,key,value,ts) VALUES (?,?,?,?)",
        (user_id, key, value, int(time.time()))
    )
    conn.commit()

def get_facts(user_id: str):
    rows = cur.execute("SELECT key, value, ts FROM user_facts WHERE user_id=?", (user_id,)).fetchall()
    return {k: v for (k, v, _) in rows}

upsert_fact("carlos", "nombre", "Carlos")
upsert_fact("carlos", "profesion", "marketing digital")
print(get_facts("carlos"))


{'nombre': 'Carlos', 'profesion': 'marketing digital'}


## - Storage + Indexing (memoria semántica) con Vector Store (Chroma)

In [6]:
%pip -q install sentence-transformers

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

emb = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    encode_kwargs={"normalize_embeddings": True},
) # Semantic representation

vectordb = Chroma(
    collection_name="memoria_conversaciones",
    embedding_function=emb,
    persist_directory=None  # in-memory. Change this to be persistent
)

vectordb.add_documents([
    Document(page_content="Carlos quiere mejorar SEO.", metadata={"user_id":"carlos","ts":1}),
    Document(page_content="Carlos prefiere cursos con proyectos prácticos.", metadata={"user_id":"carlos","ts":2}),
    Document(page_content="Carlos está interesado en analítica web.", metadata={"user_id":"carlos","ts":3}),
])

print("✅ Docs indexados:", vectordb._collection.count())


/tmp/ipython-input-118650543.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-118650543.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(


✅ Docs indexados: 3


## - Retrieval: traer recuerdos relevantes (filtrando por usuario)


In [8]:

q = "¿Qué le interesa aprender a Carlos?" # se convierte en un embedding que permite buscar
hits = vectordb.similarity_search(q, k=2, filter={"user_id":"carlos"})
for d in hits:
    print("-", d.page_content)


- Carlos está interesado en analítica web.
- Carlos prefiere cursos con proyectos prácticos.


## - Extraction: extraer hechos valiosos con salida estructurada


In [9]:

from pydantic import BaseModel, Field
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate

class ExtractedFacts(BaseModel):
    nombre: Optional[str] = Field(default=None)
    profesion: Optional[str] = Field(default=None)
    preferencia_aprendizaje: Optional[str] = Field(default=None)

extract_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Extrae SOLO hechos estables del usuario para memoria a largo plazo. "
     "No incluyas saludos ni información temporal. Devuelve SOLO JSON."),
    ("user", "Conversación:\n{conversation}")
])

extract_chain = extract_prompt | llm.with_structured_output(ExtractedFacts)

conversation = """
Usuario: Hola, soy Carlos. Trabajo en marketing digital.
Usuario: Me gustan cursos con proyectos prácticos y casos reales.
Usuario: Esta semana estoy estresado por una entrega.
"""

facts = extract_chain.invoke({"conversation": conversation})
print(facts)


nombre='Carlos' profesion='marketing digital' preferencia_aprendizaje=None


In [10]:

for k, v in facts.model_dump().items():
    if v:
        upsert_fact("carlos", k, v)

print("Hechos persistidos:", get_facts("carlos"))


Hechos persistidos: {'nombre': 'Carlos', 'profesion': 'marketing digital'}


## - LangChain: responder usando hechos + recuerdos recuperados


In [11]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import json

assistant_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un asistente que recomienda cursos.\n"
     "Hechos (estructurados): {facts}\n"
     "Recuerdos (semánticos):\n{memories}\n"),
    ("user", "{question}")
])

def answer_with_ltm(user_id: str, question: str):
    f = get_facts(user_id)
    sem = vectordb.similarity_search(question, k=2, filter={"user_id": user_id})
    memories = "\n".join([f"- {d.page_content}" for d in sem]) if sem else "(sin recuerdos)"
    chain = assistant_prompt | llm | StrOutputParser()
    return chain.invoke({"facts": json.dumps(f, ensure_ascii=False), "memories": memories, "question": question})

print(answer_with_ltm("carlos", "¿Qué cursos me recomiendas para mi profesión?"))


Considerando tu profesión en marketing digital y tus intereses en analítica web y proyectos prácticos, te recomiendo los siguientes cursos:

1. **Google Analytics Academy**: Este curso te enseñará a utilizar Google Analytics para medir y analizar el comportamiento de los usuarios en tu sitio web. Es un curso práctico y gratuito que te permitirá mejorar tus habilidades en analítica web.
2. **Certificación en Marketing Digital de HubSpot**: Este curso te enseñará a crear estrategias de marketing efectivas y a utilizar herramientas de marketing digital como HubSpot. Incluye proyectos prácticos y te certificarás al finalizar.
3. **Cours en Data Science con Python de DataCamp**: Si estás interesado en analítica web y quieres aprender a trabajar con datos, este curso te enseñará a utilizar Python para analizar y visualizar datos. Incluye proyectos prácticos y te permitirá mejorar tus habilidades en data science.
4. **Certificación en SEO de Ahrefs**: Si estás interesado en mejorar la visibil

## 4) LangGraph: checkpointing (memoria de sesión persistente)

Checkpointing persiste el **estado** del grafo (corto plazo entre ejecuciones).


In [12]:

from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

class ChatState(TypedDict):
    messages: List[BaseMessage]

def chat_node(state: ChatState):
    resp = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [AIMessage(content=resp.content)]}

g = StateGraph(ChatState)
g.add_node("chat", chat_node)
g.set_entry_point("chat")
g.add_edge("chat", END)

app = g.compile(checkpointer=MemorySaver())
thread_id = "thread-carlos-1"

out1 = app.invoke({"messages":[HumanMessage(content="Me llamo Carlos y trabajo en marketing digital.")]},
                  config={"configurable":{"thread_id": thread_id}})
print(out1["messages"][-1].content)

out2 = app.invoke({"messages":[HumanMessage(content="¿Qué cursos me recomiendas para mi profesión?")]},
                  config={"configurable":{"thread_id": thread_id}})
print(out2["messages"][-1].content)


Hola Carlos, ¿cómo estás? Me alegra conocerte. ¿En qué área específica del marketing digital trabajas? ¿En SEO, publicidad en redes sociales, creación de contenido o algo más?
Para recomendarte cursos adecuados, necesitaría saber más sobre tu profesión y objetivos. Sin embargo, te puedo sugerir algunas categorías de cursos que podrían ser útiles para diferentes profesiones:

1. **Desarrollo de habilidades digitales**:
 * Programación (Python, Java, JavaScript, etc.)
 * Desarrollo web (HTML, CSS, React, Angular, etc.)
 * Análisis de datos (Excel, Power BI, Tableau, etc.)
 * Marketing digital (SEO, publicidad en línea, etc.)
2. **Comunicación y liderazgo**:
 * Comunicación efectiva
 * Liderazgo y gestión de equipos
 * Resolución de conflictos
 * Presentación y oratoria
3. **Negocios y finanzas**:
 * Contabilidad y finanzas
 * Gestión de proyectos
 * Marketing y ventas
 * Economía y política
4. **Salud y bienestar**:
 * Primeros auxilios y atención médica
 * Salud mental y bienestar
 * Nu

## 5) Estrategia híbrida en LangGraph (Retrieval + Chat)

Checkpointing + almacenamiento externo de hechos + retrieval semántico.


In [13]:

from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
import json

class HybridState(TypedDict):
    user_id: str
    messages: List[BaseMessage]
    facts: Dict[str, Any]
    memories: str

def retrieval_node(state: HybridState):
    uid = state["user_id"]
    f = get_facts(uid)
    q = next((m.content for m in reversed(state["messages"]) if isinstance(m, HumanMessage)), "")
    sem = vectordb.similarity_search(q, k=2, filter={"user_id": uid})
    mem = "\n".join([f"- {d.page_content}" for d in sem]) if sem else "(sin recuerdos)"
    return {"facts": f, "memories": mem}

def chat_node2(state: HybridState):
    system = SystemMessage(content=(
        "Usa estos datos para personalizar:\n"
        f"Hechos: {json.dumps(state['facts'], ensure_ascii=False)}\n"
        f"Memorias:\n{state['memories']}\n"
    ))
    resp = llm.invoke([system] + state["messages"])
    return {"messages": state["messages"] + [AIMessage(content=resp.content)]}

h = StateGraph(HybridState)
h.add_node("retrieve", retrieval_node)
h.add_node("chat", chat_node2)
h.set_entry_point("retrieve")
h.add_edge("retrieve","chat")
h.add_edge("chat", END)

app2 = h.compile(checkpointer=MemorySaver())

print(app2.invoke(
    {"user_id":"carlos","messages":[HumanMessage(content="Recomiéndame cursos para mi profesión.")], "facts":{}, "memories":""},
    config={"configurable":{"thread_id":"thread-carlos-2"}}
)["messages"][-1].content)


¡Claro! Considerando tu profesión en marketing digital y tus preferencias, te recomiendo los siguientes cursos:

1. **SEO para principiantes**: Un curso práctico que te enseñará a mejorar la posición de tu sitio web en los motores de búsqueda, aumentar la visibilidad y atraer más tráfico.
2. **Marketing digital especializado en SEO**: Un curso más avanzado que te enseñará a utilizar herramientas y técnicas para mejorar la optimización de motores de búsqueda, analizar y mejorar la experiencia del usuario.
3. **Google Analytics y Google Tag Manager**: Un curso que te enseñará a utilizar herramientas de análisis y seguimiento para medir el rendimiento de tus campañas de marketing y mejorar la experiencia del usuario.
4. **Crea campañas de marketing digital efectivas**: Un curso que te enseñará a planificar y ejecutar campañas de marketing digital que generen resultados y aumenten la conversión.
5. **Marketing digital en redes sociales**: Un curso que te enseñará a utilizar las redes socia